# Part 2, Topic 3: Voltage Glitching to Dump Memory (MAIN)

---
NOTE: This lab references some (commercial) training material on [ChipWhisperer.io](https://www.ChipWhisperer.io). You can freely execute and use the lab per the open-source license (including using it in your own courses if you distribute similarly), but you must maintain notice about this source location. Consider joining our training course to enjoy the full experience.

---

**SUMMARY:** *In the previous labs, we learned how voltage glitching can be used for a similar function as clock glitching. We also learned about how it has fewer limitations, but can be less reliable for certain target setups. It also changes a great deal based on the properties of the glitch circuit itself - even changing a wire can have a huge effect.*

*In this lab, we'll use what we learned in the last lab to again attack the vulnerable serial printing of the bootloader*

**LEARNING OUTCOMES:**

* Applying previous glitch settings to new firmware
* Checking for success and failure when glitching
* Understanding how compiler optimizations can cause devices to behave in strange ways

## The Situation

You should already know the situation from your previous attempts at glitching this bootloader (as well as what the flaw is). No need to do big long searches for parameters, just use values that worked well for the previous tutorial.

In [ ]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEXMEGA'

In [ ]:
%%bash -s "$PLATFORM"
cd ../../../hardware/victims/firmware/bootloader-glitch
make PLATFORM=$1 CRYPTO_TARGET=NONE

In [ ]:
%run "../../Helper_Scripts/Setup_Generic.ipynb"

In [ ]:
fw_path = "../../../hardware/victims/firmware/bootloader-glitch/bootloader-{}.hex".format(PLATFORM)

In [ ]:
cw.program_target(scope, prog, fw_path)

In [ ]:
#Basic setup
# ###################
# Add your code here
# ###################
raise NotImplementedError("Add your code here, and delete this.")

In [ ]:
# GlitchController will be part of ChipWhisperer core - just run this block
# for now.


import matplotlib.pylab as plt
  
import chipwhisperer.common.results.glitch as glitch
      
gc = glitch.GlitchController(groups=["success", "reset", "normal"], parameters=["width", "offset"])
gc.display_stats()


Now the rest is up to you! 

In [ ]:
from importlib import reload
import chipwhisperer.common.results.glitch as glitch
from tqdm.notebook import tqdm
import re
import struct
# ###################
# Add your code here
# ###################
raise NotImplementedError("Add your code here, and delete this.")

In [ ]:
scope.dis()
target.dis()